# Runnable కాన్సెప్ట్

ఈ ట్యుటోరియల్‌లో, మనం LangChain లో Runnable కాన్సెప్ట్ గురించి నేర్చుకుంటాము, ఇది LCEL (LangChain Expression Language) యొక్క కోర్ బిల్డింగ్ బ్లాక్.

## సెటప్

మొదట, మనం అవసరమైన లైబ్రరీలను ఇన్‌స్టాల్ చేసుకుందాం:

In [ ]:
# అవసరమైన లైబ్రరీలను ఇన్‌స్టాల్ చేయడం
!pip install langchain langchain-openai openai

ఇప్పుడు, మనం OpenAI API కీని సెట్ చేద్దాం:

In [ ]:
import os
from dotenv import load_dotenv

# .env ఫైల్ నుండి API కీని లోడ్ చేయడం
load_dotenv()

# లేదా డైరెక్ట్‌గా సెట్ చేయడం (డెవలప్‌మెంట్ కోసం మాత్రమే, ప్రొడక్షన్‌లో ఉపయోగించవద్దు)
# os.environ["OPENAI_API_KEY"] = "మీ-API-కీ-ఇక్కడ-పెట్టండి"

## Runnable అంటే ఏమిటి?

LangChain లో, `Runnable` అనేది ఒక ఇంటర్ఫేస్ (అబ్స్ట్రాక్ట్ క్లాస్), ఇది ఇన్‌పుట్‌ని తీసుకుని ఔట్‌పుట్‌ని రిటర్న్ చేసే కంపోనెంట్‌లను డిఫైన్ చేస్తుంది. ఇది LCEL (LangChain Expression Language) యొక్క కోర్ బిల్డింగ్ బ్లాక్.

Runnable ఇంటర్ఫేస్ ఈ క్రింది మెథడ్‌లను అందిస్తుంది:

1. **invoke()**: ఒక ఇన్‌పుట్‌ని ప్రాసెస్ చేసి ఔట్‌పుట్‌ని రిటర్న్ చేస్తుంది
2. **stream()**: ఇన్‌పుట్‌ని ప్రాసెస్ చేసి ఔట్‌పుట్‌ని స్ట్రీమ్‌గా రిటర్న్ చేస్తుంది
3. **batch()**: అనేక ఇన్‌పుట్‌లను ఒకేసారి ప్రాసెస్ చేస్తుంది
4. **ainvoke()**, **astream()**, **abatch()**: అసింక్రోనస్ వెర్షన్‌లు

LangChain లో చాలా కంపోనెంట్‌లు (LLMs, ప్రాంప్ట్ టెంప్లేట్స్, ఔట్‌పుట్ పార్సర్స్, మొదలైనవి) Runnable ఇంటర్ఫేస్‌ని ఇంప్లిమెంట్ చేస్తాయి, ఇది వాటిని LCEL పైప్‌లైన్‌లలో ఉపయోగించడానికి అనుమతిస్తుంది.

## బిల్ట్-ఇన్ Runnables

LangChain లో అనేక బిల్ట్-ఇన్ Runnables ఉన్నాయి. కొన్ని ముఖ్యమైన వాటిని చూద్దాం:

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# LLM (Runnable ఇంప్లిమెంట్ చేస్తుంది)
llm = ChatOpenAI(model="gpt-3.5-turbo")

# ప్రాంప్ట్ టెంప్లేట్ (Runnable ఇంప్లిమెంట్ చేస్తుంది)
prompt = ChatPromptTemplate.from_template("తెలుగులో {topic} గురించి చిన్న వ్యాసం రాయండి.")

# ఔట్‌పుట్ పార్సర్ (Runnable ఇంప్లిమెంట్ చేస్తుంది)
output_parser = StrOutputParser()

# ప్రాంప్ట్ టెంప్లేట్‌ని invoke() చేయడం
prompt_result = prompt.invoke({"topic": "IPL క్రికెట్"})
print("ప్రాంప్ట్ టెంప్లేట్ ఔట్‌పుట్:")
print(prompt_result)
print()

# LLM ని invoke() చేయడం
llm_result = llm.invoke(prompt_result)
print("LLM ఔట్‌పుట్:")
print(llm_result)
print()

# ఔట్‌పుట్ పార్సర్‌ని invoke() చేయడం
parser_result = output_parser.invoke(llm_result)
print("ఔట్‌పుట్ పార్సర్ ఔట్‌పుట్:")
print(parser_result)

## Runnable ఇంటర్ఫేస్ మెథడ్స్

Runnable ఇంటర్ఫేస్ అందించే వివిధ మెథడ్‌లను చూద్దాం:

In [ ]:
# invoke() - సింగిల్ ఇన్‌పుట్‌ని ప్రాసెస్ చేయడం
result = llm.invoke("తెలుగు సినిమా ఇండస్ట్రీ గురించి చిన్న పరిచయం ఇవ్వండి.")
print("invoke() ఔట్‌పుట్:")
print(result.content)
print()

# stream() - ఔట్‌పుట్‌ని స్ట్రీమ్ చేయడం
print("stream() ఔట్‌పుట్:")
for chunk in llm.stream("తెలుగు క్రికెట్ ఆటగాళ్ల గురించి చిన్న పరిచయం ఇవ్వండి."):
    print(chunk.content, end="")
print("\n")

# batch() - అనేక ఇన్‌పుట్‌లను ఒకేసారి ప్రాసెస్ చేయడం
batch_inputs = [
    "తెలుగు సినిమాలలో టాప్ 3 హీరోలు ఎవరు?",
    "తెలుగు సినిమాలలో టాప్ 3 హీరోయిన్లు ఎవరు?"
]
batch_results = llm.batch(batch_inputs)
print("batch() ఔట్‌పుట్:")
for i, result in enumerate(batch_results):
    print(f"ఇన్‌పుట్ {i+1}: {batch_inputs[i]}")
    print(f"ఔట్‌పుట్: {result.content}\n")

## కస్టమ్ Runnable క్రియేట్ చేయడం

మనం స్వంత Runnable క్లాస్‌ని క్రియేట్ చేయవచ్చు, ఇది Runnable ఇంటర్ఫేస్‌ని ఇంప్లిమెంట్ చేస్తుంది:

In [ ]:
from langchain_core.runnables import Runnable
from typing import Dict, Any, List, Optional

class TeluguTextAnalyzer(Runnable):
    """తెలుగు టెక్స్ట్‌ని అనలైజ్ చేసే కస్టమ్ Runnable."""
    
    def __init__(self):
        pass
    
    def invoke(self, input: str, config: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        """తెలుగు టెక్స్ట్‌ని అనలైజ్ చేసి, బేసిక్ స్టాటిస్టిక్స్‌ని రిటర్న్ చేస్తుంది."""
        # వర్డ్ కౌంట్
        words = input.split()
        word_count = len(words)
        
        # క్యారెక్టర్ కౌంట్ (స్పేసెస్ లేకుండా)
        char_count = len(input.replace(" ", ""))
        
        # సెంటెన్స్ కౌంట్ (సింపుల్ అప్రోక్సిమేషన్)
        sentence_count = input.count(".") + input.count("!") + input.count("?") + input.count("।")
        if sentence_count == 0 and len(input) > 0:
            sentence_count = 1
        
        # అత్యంత తరచుగా వాడే పదాలు
        word_freq = {}
        for word in words:
            word = word.strip(".,!?()[]{}'\"").lower()
            if word:
                word_freq[word] = word_freq.get(word, 0) + 1
        
        # టాప్ 5 పదాలు
        top_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:5]
        
        return {
            "word_count": word_count,
            "char_count": char_count,
            "sentence_count": sentence_count,
            "top_words": top_words
        }
    
    def batch(self, inputs: List[str], config: Optional[Dict[str, Any]] = None) -> List[Dict[str, Any]]:
        """అనేక ఇన్‌పుట్‌లను ఒకేసారి ప్రాసెస్ చేస్తుంది."""
        return [self.invoke(input) for input in inputs]

# కస్టమ్ Runnable ని క్రియేట్ చేయడం
analyzer = TeluguTextAnalyzer()

# కస్టమ్ Runnable ని ఉపయోగించడం
telugu_text = """తెలుగు భారతదేశంలోని ద్రావిడ భాషలలో ఒకటి. ఇది ఆంధ్రప్రదేశ్ మరియు తెలంగాణ రాష్ట్రాల అధికారిక భాష. తెలుగు భాష చాలా మధురమైన భాష. దీనిని 'ఇటలీ ఆఫ్ ది ఈస్ట్' అని కూడా పిలుస్తారు. తెలుగు సాహిత్యం చాలా సమృద్ధిగా ఉంది. ననయ్య, తిక్కన, ఎర్రన వంటి కవులు తెలుగు సాహిత్యానికి చాలా సేవ చేశారు. తెలుగు సినిమా పరిశ్రమ కూడా చాలా పెద్దది. ఇది భారతదేశంలోని అతిపెద్ద సినిమా పరిశ్రమలలో ఒకటి."""

result = analyzer.invoke(telugu_text)
print("తెలుగు టెక్స్ట్ అనాలిసిస్:")
print(f"పదాల సంఖ్య: {result['word_count']}")
print(f"అక్షరాల సంఖ్య: {result['char_count']}")
print(f"వాక్యాల సంఖ్య: {result['sentence_count']}")
print("అత్యంత తరచుగా వాడే పదాలు:")
for word, count in result['top_words']:
    print(f"  {word}: {count} సార్లు")

## Runnable లను కలపడం

Runnable లను LCEL ఉపయోగించి కలపవచ్చు:

In [ ]:
# ప్రాంప్ట్ టెంప్లేట్ క్రియేట్ చేయడం
prompt = ChatPromptTemplate.from_template("తెలుగులో {topic} గురించి ఒక పేరా రాయండి.")

# LLM క్రియేట్ చేయడం
llm = ChatOpenAI(model="gpt-3.5-turbo")

# ఔట్‌పుట్ పార్సర్ క్రియేట్ చేయడం
output_parser = StrOutputParser()

# టెక్స్ట్ అనలైజర్ క్రియేట్ చేయడం
analyzer = TeluguTextAnalyzer()

# Runnable లను కలపడం
chain = prompt | llm | output_parser | analyzer

# చెయిన్‌ని రన్ చేయడం
result = chain.invoke({"topic": "తెలుగు సినిమా సంగీతం"})
print("తెలుగు సినిమా సంగీతం గురించి టెక్స్ట్ అనాలిసిస్:")
print(f"పదాల సంఖ్య: {result['word_count']}")
print(f"అక్షరాల సంఖ్య: {result['char_count']}")
print(f"వాక్యాల సంఖ్య: {result['sentence_count']}")
print("అత్యంత తరచుగా వాడే పదాలు:")
for word, count in result['top_words']:
    print(f"  {word}: {count} సార్లు")

## RunnableConfig

Runnable లను రన్ చేసేటప్పుడు, మనం `RunnableConfig` ఉపయోగించి వివిధ ఆప్షన్‌లను కాన్ఫిగర్ చేయవచ్చు:

In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain.callbacks.base import BaseCallbackHandler
from typing import Dict, Any, List

# కస్టమ్ కాల్‌బ్యాక్ హ్యాండ్లర్ క్రియేట్ చేయడం
class TeluguLoggingCallbackHandler(BaseCallbackHandler):
    """ప్రాసెసింగ్ స్టెప్స్‌ని లాగ్ చేసే కస్టమ్ కాల్‌బ్యాక్ హ్యాండ్లర్."""
    
    def on_llm_start(self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any) -> None:
        """LLM స్టార్ట్ అయినప్పుడు కాల్ చేయబడుతుంది."""
        print("🚀 LLM ప్రాసెసింగ్ స్టార్ట్ అయింది...")
        for i, prompt in enumerate(prompts):
            print(f"ప్రాంప్ట్ {i+1}: {prompt[:50]}...")
    
    def on_llm_end(self, response, **kwargs: Any) -> None:
        """LLM పూర్తయినప్పుడు కాల్ చేయబడుతుంది."""
        print("✅ LLM ప్రాసెసింగ్ పూర్తయింది!")
    
    def on_chain_start(self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any) -> None:
        """చెయిన్ స్టార్ట్ అయినప్పుడు కాల్ చేయబడుతుంది."""
        print(f"⛓️ చెయిన్ స్టార్ట్ అయింది: {serialized.get('name', 'Unknown')}")
    
    def on_chain_end(self, outputs: Dict[str, Any], **kwargs: Any) -> None:
        """చెయిన్ పూర్తయినప్పుడు కాల్ చేయబడుతుంది."""
        print("⛓️ చెయిన్ పూర్తయింది!")

# కాల్‌బ్యాక్ హ్యాండ్లర్‌ని క్రియేట్ చేయడం
callback_handler = TeluguLoggingCallbackHandler()

# RunnableConfig క్రియేట్ చేయడం
config = RunnableConfig(
    callbacks=[callback_handler],
    tags=["తెలుగు", "సినిమా"],
    metadata={"language": "తెలుగు", "topic": "సినిమా"},
    verbose=True
)

# ప్రాంప్ట్ టెంప్లేట్ క్రియేట్ చేయడం
prompt = ChatPromptTemplate.from_template("తెలుగు సినిమా {movie_name} గురించి ఒక చిన్న సమీక్ష రాయండి.")

# LLM క్రియేట్ చేయడం
llm = ChatOpenAI(model="gpt-3.5-turbo")

# చెయిన్ క్రియేట్ చేయడం
chain = prompt | llm

# కాన్ఫిగరేషన్‌తో చెయిన్‌ని రన్ చేయడం
response = chain.invoke({"movie_name": "RRR"}, config=config)
print("\nఫైనల్ రెస్పాన్స్:")
print(response.content)

## Runnable లతో రియల్-వరల్డ్ ఉదాహరణ: తెలుగు న్యూస్ సమ్మరైజర్

ఇప్పుడు, మనం Runnable లను ఉపయోగించి ఒక రియల్-వరల్డ్ ఉదాహరణను చూద్దాం: తెలుగు న్యూస్ ఆర్టికల్స్‌ని సమ్మరైజ్ చేయడం.

In [ ]:
from langchain_core.runnables import Runnable, RunnablePassthrough
from typing import Dict, Any, List, Optional

class TeluguNewsSummarizer(Runnable):
    """తెలుగు న్యూస్ ఆర్టికల్స్‌ని సమ్మరైజ్ చేసే కస్టమ్ Runnable."""
    
    def __init__(self, llm):
        self.llm = llm
        self.summary_prompt = ChatPromptTemplate.from_template(
            """కింది తెలుగు న్యూస్ ఆర్టికల్‌ని చదివి, 1-2 వాక్యాలలో సంక్షిప్తంగా సమ్మరైజ్ చేయండి:
            
            {article}
            
            సమ్మరీ:
            """
        )
        self.keywords_prompt = ChatPromptTemplate.from_template(
            """కింది తెలుగు న్యూస్ ఆర్టికల్‌ని చదివి, 5-7 కీలక పదాలను (keywords) గుర్తించండి:
            
            {article}
            
            కీలక పదాలు:
            """
        )
        self.output_parser = StrOutputParser()
    
    def invoke(self, input: Dict[str, Any], config: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        """న్యూస్ ఆర్టికల్‌ని సమ్మరైజ్ చేసి, కీలక పదాలను గుర్తిస్తుంది."""
        article = input["article"]
        
        # సమ్మరీ జనరేట్ చేయడం
        summary_chain = self.summary_prompt | self.llm | self.output_parser
        summary = summary_chain.invoke({"article": article}, config=config)
        
        # కీలక పదాలను గుర్తించడం
        keywords_chain = self.keywords_prompt | self.llm | self.output_parser
        keywords = keywords_chain.invoke({"article": article}, config=config)
        
        # అనలైజర్‌ని ఉపయోగించి అదనపు స్టాటిస్టిక్స్‌ని పొందడం
        analyzer = TeluguTextAnalyzer()
        stats = analyzer.invoke(article)
        
        return {
            "original_article": article,
            "summary": summary,
            "keywords": keywords,
            "word_count": stats["word_count"],
            "sentence_count": stats["sentence_count"]
        }

# తెలుగు న్యూస్ ఆర్టికల్స్
telugu_news_articles = [
    """
    హైదరాబాద్, మే 15: ఐపీఎల్ 2024లో సన్‌రైజర్స్ హైదరాబాద్ జట్టు అద్భుతమైన ప్రదర్శనతో అభిమానులను ఆకట్టుకుంటోంది. గత మ్యాచ్‌లో లక్నో సూపర్ జెయింట్స్‌పై 10 వికెట్ల తేడాతో ఘన విజయం సాధించిన సన్‌రైజర్స్, పాయింట్ల పట్టికలో రెండో స్థానానికి చేరుకుంది.
    
    ఈ మ్యాచ్‌లో ట్రావిస్ హెడ్ (89*) మరియు అభిషేక్ శర్మ (75*) అద్భుతమైన బ్యాటింగ్ ప్రదర్శనతో జట్టును విజయతీరాలకు చేర్చారు. లక్నో జట్టు నిర్దేశించిన 166 పరుగుల లక్ష్యాన్ని సన్‌రైజర్స్ కేవలం 9.4 ఓవర్లలోనే చేధించి, టీ20 క్రికెట్‌లో అత్యంత వేగవంతమైన చేజింగ్‌లలో ఒకటిగా నిలిచింది.
    """,
    """
    హైదరాబాద్, మే 16: తెలంగాణ ప్రభుత్వం రాష్ట్రంలో కొత్త విద్యా విధానాన్ని ప్రవేశపెట్టనుంది. ఈ విధానంలో డిజిటల్ లెర్నింగ్‌కు ప్రాధాన్యత ఇవ్వనున్నారు. రాష్ట్ర విద్యా శాఖ మంత్రి మాట్లాడుతూ, కొత్త విద్యా విధానం ద్వారా విద్యార్థుల నైపుణ్యాలను మెరుగుపరచడమే లక్ష్యమని తెలిపారు.
    
    ప్రతి పాఠశాలలో డిజిటల్ క్లాస్‌రూమ్‌లను ఏర్పాటు చేయడం, ఉపాధ్యాయులకు ప్రత్యేక శిక్షణ ఇవ్వడం, ఆన్‌లైన్ లెర్నింగ్ ప్లాట్‌ఫామ్‌లను అందించడం వంటి చర్యలు ఈ విధానంలో భాగంగా ఉన్నాయి. ఈ కొత్త విద్యా విధానం వచ్చే విద్యా సంవత్సరం నుండి అమలులోకి రానుంది.
    """
]

# LLM క్రియేట్ చేయడం
llm = ChatOpenAI(model="gpt-3.5-turbo")

# న్యూస్ సమ్మరైజర్ క్రియేట్ చేయడం
summarizer = TeluguNewsSummarizer(llm)

# ప్రతి న్యూస్ ఆర్టికల్‌ని సమ్మరైజ్ చేయడం
for i, article in enumerate(telugu_news_articles):
    print(f"న్యూస్ ఆర్టికల్ {i+1} సమ్మరీ:")
    result = summarizer.invoke({"article": article})
    print(f"సమ్మరీ: {result['summary']}")
    print(f"కీలక పదాలు: {result['keywords']}")
    print(f"పదాల సంఖ్య: {result['word_count']}")
    print(f"వాక్యాల సంఖ్య: {result['sentence_count']}")
    print("\n" + "-"*50 + "\n")

## ముగింపు

ఈ ట్యుటోరియల్‌లో, మనం LangChain లో Runnable కాన్సెప్ట్ గురించి నేర్చుకున్నాము:

1. Runnable అంటే ఏమిటో మరియు దాని ప్రాముఖ్యత గురించి తెలుసుకున్నాము
2. బిల్ట్-ఇన్ Runnables గురించి నేర్చుకున్నాము
3. Runnable ఇంటర్ఫేస్ మెథడ్స్ (invoke, stream, batch) గురించి చూశాము
4. కస్టమ్ Runnable ని ఎలా క్రియేట్ చేయాలో నేర్చుకున్నాము
5. Runnable లను ఎలా కలపాలో చూశాము
6. RunnableConfig ఎలా ఉపయోగించాలో నేర్చుకున్నాము
7. Runnable లతో రియల్-వరల్డ్ ఉదాహరణను ఇంప్లిమెంట్ చేశాము

Runnable అనేది LangChain లో చాలా ముఖ్యమైన కాన్సెప్ట్, ఇది LCEL (LangChain Expression Language) యొక్క కోర్ బిల్డింగ్ బ్లాక్. ఇది మనకు LLM అప్లికేషన్‌లను మాడ్యులర్ మరియు కంపోజబుల్ వేలో బిల్డ్ చేయడానికి అనుమతిస్తుంది.

తదుపరి ట్యుటోరియల్‌లో, మనం ప్రాంప్ట్ ఇంజినీరింగ్ టెక్నిక్స్ గురించి నేర్చుకుంటాము.